In [48]:
from pyspark.sql import *
import getpass
username = getpass.getuser()
spark = SparkSession.builder \
    .appName('loans_defaulter') \
    .config('spark.ui.port', '0') \
    .config('spark.shuffle.useOldFetchProtocol','true') \
    .config("spark.sql.warehouse.dir", f"/user/{username}/warehouse") \
    .enableHiveSupport() \
    .master('yarn') \
    .getOrCreate()

In [47]:
spark.stop()

In [2]:
spark

In [3]:
spark.sql("use itv_012667_lending_club")

""


In [4]:
spark.sql("show tables")

database,tableName,isTemporary
itv_012667_lendin...,customer_data,false
itv_012667_lendin...,loan_repayments,false
itv_012667_lendin...,loans,false
itv_012667_lendin...,loans_defaulters_...,false
itv_012667_lendin...,loans_defaulters_...,false


In [9]:
bad_data_customer_df = spark.sql("select member_id from itv_012667_lending_club.customer_data group by member_id having count(member_id) > 1")

In [11]:
bad_data_customer_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv012667/bad/bad_customer_data").save()

In [12]:
bad_data_loans_defaulters_delinq_df = spark.sql("select member_id from loans_defaulters_delinq group by member_id having count(member_id)>1")

In [14]:
bad_data_loans_defaulters_delinq_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv012667/bad/bad_data_loans_defaulters_delinq").save()

In [15]:
bad_data_loans_defaulters_detail_rec_enq_df = spark.sql("select member_id from loans_defaulters_detail_rec_enq group by member_id having count(member_id)>1")

In [17]:
bad_data_loans_defaulters_detail_rec_enq_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv012667/bad/bad_data_loans_defaulters_detail_rec_enq").save()

In [23]:
bad_data_members = bad_data_customer_df.select("member_id").union(bad_data_loans_defaulters_delinq_df).select("member_id").union(bad_data_loans_defaulters_detail_rec_enq_df).select("member_id")

In [24]:
bad_data_members_final = bad_data_members.distinct()

In [25]:
bad_data_members_final.count()

3175

In [26]:
bad_customer_data_final_df = spark.read.csv("/user/itv012667/lendingclub/bad/bad/bad_customer_data_final",header=True,inferSchema=True)

In [27]:
bad_customer_data_final_df.createOrReplaceTempView("bad_data_customer")

In [28]:
spark.sql("select * from bad_data_customer")

member_id
514da6011e2f772fb...
d3125d00da326895a...
4e9cc2faea777e410...
8d42ec724248a79d2...
c14309ba2311e430d...
d9ce4046daf599732...
e0ecccbcb94183be6...
fecada6499caf7a4d...
62d93eb316a3eacf4...
e757d51dddd8fa34e...


In [32]:
customers_df = spark.sql("select * from customer_data where member_id not in (select * from bad_data_customer) ")

In [35]:
customers_df.write.mode("overwrite").option("path","/user/itv012667/lendingclub/cleaned_new/customers_parquet").save()

In [49]:
loan_delinq = spark.read.csv("/user/itv012667/lendingclub/bad/bad/bad_data_loans_defaulters_delinq",inferSchema=True,header=True)

In [52]:
loan_delinq.createOrReplaceTempView("loan_delinq")

In [54]:
loan_defaulters_delinq_df = spark.sql("select * from itv_012667_lending_club.loans_defaulters_delinq where member_id not in (select * from loan_delinq)")

In [55]:
loan_defaulters_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,0
aac68850fdac09fd0...,1,0.0,0
c89986155a070db2e...,1,0.0,15
4e1c30a5dfe9f1e20...,0,0.0,34
1eef79a0e79b72c7a...,1,0.0,0
ec1953dba2cfb89ad...,2,0.0,0
3712c9da85e54b7b1...,1,0.0,0
030e5c9c411c936c5...,2,0.0,0
252736f8401b6b046...,2,0.0,0
2d73c65a7b7d1a351...,1,0.0,0


In [56]:
loan_defaulters_delinq_df.write.mode("overwrite").option("path","/user/itv012667/lendingclub/cleaned_new/loan_defaulters_delinq_parquet").save()

In [57]:
loan_defaulters_delinq_df.show()

+--------------------+-----------+-----------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|mths_since_last_delinq|
+--------------------+-----------+-----------+----------------------+
|9cb79aa7323e81be1...|          2|        0.0|                     0|
|aac68850fdac09fd0...|          1|        0.0|                     0|
|c89986155a070db2e...|          1|        0.0|                    15|
|4e1c30a5dfe9f1e20...|          0|        0.0|                    34|
|1eef79a0e79b72c7a...|          1|        0.0|                     0|
|ec1953dba2cfb89ad...|          2|        0.0|                     0|
|3712c9da85e54b7b1...|          1|        0.0|                     0|
|030e5c9c411c936c5...|          2|        0.0|                     0|
|252736f8401b6b046...|          2|        0.0|                     0|
|2d73c65a7b7d1a351...|          1|        0.0|                     0|
|4c5cc017a06ad9cee...|          1|        0.0|                     0|
|b2ae51f7010bda712..

In [41]:
spark.sql("show tables").show(truncate=False)

+-----------------------+-------------------------------+-----------+
|database               |tableName                      |isTemporary|
+-----------------------+-------------------------------+-----------+
|itv_012667_lending_club|customer_data                  |false      |
|itv_012667_lending_club|loan_repayments                |false      |
|itv_012667_lending_club|loans                          |false      |
|itv_012667_lending_club|loans_defaulters_delinq        |false      |
|itv_012667_lending_club|loans_defaulters_detail_rec_enq|false      |
|                       |bad_data_customer              |true       |
|                       |loan_delinq                    |true       |
+-----------------------+-------------------------------+-----------+



In [59]:
loans_defaulters_detail_rec_enq_df

member_id
5a20222639d436b66...
76b577467eda5bdbc...
4273588c35c7b7f40...
bbe43331566910d55...
844149bec10005cd7...
827a048c072fb44b0...
a4327ad776a277161...
9632a7cdd49aa234e...
14a8d9dba7c66acae...
6ebda6b65772ea4f5...


In [58]:
loans_defaulters_detail_rec_enq_df = spark.read.csv("/user/itv012667/lendingclub/bad/bad/bad_data_loans_defaulters_detail_rec_enq",header=True,inferSchema=True)

In [60]:
loans_defaulters_detail_rec_enq_df.createOrReplaceTempView("loans_defaulters_detail_rec_enq_df")

In [62]:
detail_rec = spark.sql("select * from itv_012667_lending_club.loans_defaulters_detail_rec_enq where member_id not in (select * from loans_defaulters_detail_rec_enq_df)")

In [63]:
detail_rec.write.mode("overwrite").option("path","/user/itv012667/lendingclub/cleaned_new/loans_defaulters_detail_rec_enq_parquet").save()

In [77]:
spark.sql("drop table if exists itv_012667_lending_club.customers_new")

""


In [78]:
spark.sql("""
create EXTERNAL TABLE itv_012667_lending_club.customers_new(member_id string, emp_title string, emp_length int, home_ownership string, 
annual_income float, address_state string, address_zipcode string, address_country string, grade string, 
sub_grade string, verification_status string, total_high_credit_limit float, application_type string, 
join_annual_income float, verification_status_joint string, ingest_date timestamp)
stored as parquet
LOCATION '/public/trendytech/lendingclubproject/cleaned_new/customer_parquet'
""")

""


In [80]:
spark.sql("""
create EXTERNAL TABLE itv_012667_lending_club.loans_defaulters_delinq_new(member_id string,delinq_2yrs integer, delinq_amnt float, mths_since_last_delinq integer)
stored as parquet
LOCATION '/public/trendytech/lendingclubproject/cleaned_new/loan_defaulters_delinq_parquet'
""")

AnalysisException: Table or view 'loans_defaulters_delinq_new' already exists in database 'itv_012667_lending_club'

In [82]:
spark.sql("""
create EXTERNAL TABLE itv_012667_lending_club.loans_defaulters_detail_rec_enq_new(member_id string, pub_rec integer, pub_rec_bankruptcies integer, inq_last_6mths integer)
stored as parquet
LOCATION '/public/trendytech/lendingclubproject/cleaned_new/loan_defaulters_detail_rec_enq_parquet'
""")

""


In [83]:
spark.sql("use itv_012667_lending_club")

""


In [87]:
spark.sql("show tables").show(truncate=False)

+-----------------------+-----------------------------------+-----------+
|database               |tableName                          |isTemporary|
+-----------------------+-----------------------------------+-----------+
|itv_012667_lending_club|customer_data                      |false      |
|itv_012667_lending_club|customers_new                      |false      |
|itv_012667_lending_club|loan_repayments                    |false      |
|itv_012667_lending_club|loans                              |false      |
|itv_012667_lending_club|loans_defaulters_delinq            |false      |
|itv_012667_lending_club|loans_defaulters_delinq_new        |false      |
|itv_012667_lending_club|loans_defaulters_detail_rec_enq    |false      |
|itv_012667_lending_club|loans_defaulters_detail_rec_enq_new|false      |
|                       |loan_delinq                        |true       |
|                       |loans_defaulters_detail_rec_enq_df |true       |
+-----------------------+-------------

In [89]:
spark.sql("""select member_id, count(*) as total 
from loans_defaulters_detail_rec_enq_new 
group by member_id order by total desc""")

member_id,total
50e53d129bb455570...,1
07cd0b16f814b67ad...,1
89272832cfda308aa...,1
da3dd24815ec7ad41...,1
d5817b0a9d35a12c5...,1
fb4f52d5debc6d2c6...,1
2a211b6c34c6c93b6...,1
33f2a017bbd919ac3...,1
b99ed769e79b89d88...,1
44a0ae91234e7bc7e...,1
